Welcome to the ***CALINS*** uncertainty propagation module usage notebook.

# Enabling/Disabling terminal verbosity, offline outputs export, & changing the logs folder

In [ ]:
import calins as cl

# Disabling terminal verbosity
cl.logs.VERBOSE = False

# Enabling offline export of HTML files. It will add the Plotly.js scripts in the HTML files (~4.4 Mo).
cl.classes.OFFLINE_EXPORT = True

# Changing the folder containing log files. By default, the folder is $User/.CALINS/
cl.logs.LOG_DIR = "NEW/LOG/DIR"

# Preliminary analysis of the study case:

In [ ]:
import calins as cl

sensi_file_path = '/PATH/TO/SENSI/FILE/sensi.44g.sdf'

# Creating the Case object:
#   occurrences_rule: rule for handling multiple occurrences of the same iso-reac pair, here we choose "sum" to sum the sensitivities (default value)
#   you can choose "first", "last", or "sum"
cas_application = cl.Case(sdf_path=sensi_file_path, occurrences_rule="first")

# Creating an HTML file to visualize sensitivities
#   plotting_unit: unit for plotting sensitivities, here we choose "pcm", meaning [pcm]/%[XS]. Other option is "relative", meaning %[resp]/%[XS]
cas_application.export_to_html(output_html_path='PATH/TO/OUTPUT/cas_application.html', plotting_unit='pcm')

# Printing the attributes of our case:
print(cas_application.casename, cas_application.group_nb, cas_application.e_bins)
print(f"Iso-reac list: {cas_application.iso_reac_list}")
print(f"Calculated Keff: {cas_application.resp_calc} +/- {cas_application.sigma_resp_calc}")
print(f"Experimental Keff (if present): {cas_application.resp_expe} +/- {cas_application.sigma_resp_expe}")

# A priori uncertainty calculation:

In [ ]:
# USING THE SCALE MATRIX WITH 44 GROUPS

import calins as cl

scale_file_path = '/PATH/TO/SCALE/FILE/scale_44g'
sensi_file_path = '/PATH/TO/SENSI/FILE/sensi.44g.sdf'

# Recommended: Use NDCovariances object for automatic format detection and better data handling
cov_data = cl.NDCovariances(input_path=scale_file_path, format='coverx')  # or format='auto'

# Uncertainty calculation: creating a cl.Uncertainty object
#   study_case: can be a Case object or a path to an sdf file
#   cov_data: NDCovariances object
#   output_html_path: save results in an HTML file containing the sensitivity profile, cov matrix, uncertainty decomposition, etc.
unc_prior = cl.calcul_uncertainty(study_case=sensi_file_path, cov_data=cov_data, reac_list=[2, 4, 16, 18, 102, 103, 452, 1018], output_html_path='PATH/TO/OUTPUT/unc_prior.html')
print(f"The a priori uncertainty is: {unc_prior.value} pcm")
print(f"With a calculated Keff of: {unc_prior.resp_calc}")

# Save the decomposition in an .xlsx file (decomposition already included in the HTML file)
unc_prior.decomposition.to_excel('/PATH/TO/EXCEL/FILE/unc_prior_decomposition.xlsx')

# Uncertainty calculation omitting Pu-239 and including only reactions 2 and 102 (if they exist in the covariance matrix)
unc_prior_bis = cl.calcul_uncertainty(study_case=sensi_file_path, cov_data=cov_data, reac_list=[2, 102], exclude_iso=['PU-239'], output_html_path='PATH/TO/OUTPUT/unc_prior_bis.html')
print(f"The a priori uncertainty is: {unc_prior_bis.value} pcm")
print(f"With a calculated Keff of: {unc_prior_bis.resp_calc}")
print(f"The included isotope-reaction pairs are: {unc_prior_bis.iso_reac_list}")

# Save the decomposition in an .xlsx file
unc_prior_bis.decomposition.to_excel('/PATH/TO/EXCEL/FILE/unc_prior_bis_decomposition.xlsx')

# Optional: Export covariance data to Excel and reload it later
cov_data.write_xlsx('/PATH/TO/EXCEL/FILE/covariance_export.xlsx')
# Later, reload from Excel:
cov_data_reloaded = cl.NDCovariances(input_path='/PATH/TO/EXCEL/FILE/covariance_export.xlsx', format='xlsx')

# Assimilation calculation WITHOUT pre-filtering on the Ck factor, with $\chi ^2$ filter

In [ ]:
# USING THE COMAC MATRIX WITH 15 GROUPS

import calins as cl
import os

# The original formatting of COMAC matrices consists of a folder of files
comac_folder = '/PATH/TO/COMAC/FOLDER/'
case_sensi_file_path = '/PATH/TO/CASE/SENSI/FILE/case_sensi.15g.sdf'
benchmarks_folder = '/PATH/TO/BENCHMARKS/'

# Necessary: list of paths to benchmark cases
benchmarks_paths = [os.path.join(benchmarks_folder, file) for file in os.listdir(benchmarks_folder) if file.endswith('.sdf')]

# Recommended: Use NDCovariances object for automatic format detection
cov = cl.NDCovariances(input_path=comac_folder, format='comac')

# Assimilation with a Chi2 < 1.2 filter: creating a cl.Assimilation object and saving the results
# The 'output_html_path' option triggers the 'export_to_html' function, which saves all GLLSM results and parameters, with details of U238 submatrices ('isotopes_to_detail')
assim = cl.Assimilation(benchmarks_list=benchmarks_paths, 
                        study_case=case_sensi_file_path, 
                        cov_data=cov,
                        reac_list=[2, 4, 16, 18, 102, 103, 452, 1018], 
                        targetted_chi2=1.2,
                        filtering_chi2_method='diagonal', 
                        output_html_path='/PATH/TO/RESULTS/assim.html', 
                        isotopes_to_detail=['U238'])
print(f"UNC PRIOR: {assim.prior_uncertainty.value} pcm")
print(f"UNC POST: {assim.post_uncertainty.value} pcm")
print(f"BIAS: {assim.bias.value} pcm")
print(f"Keff CASE: {assim.study_case.resp_calc} +/- {assim.study_case.sigma_resp_calc}")

# Save the bias decomposition by iso-reac pair:
#assim.bias.decomposition.to_excel('/PATH/TO/EXCEL/FILE/assim_bias-decomposition_COMAC_15g.xlsx')

# Save the sensitivity histogram of the study case in HTML format
#assim.export_to_html(output_html_path='/PATH/TO/HTML/FILE/assim_case-sensi_15g.html')

# Display benchmark cases excluded by the Chi2 filter (DataFrame containing info on all benchmark cases)
print(assim.bench_list[['PATH', 'REMOVED']])

# Assimilation calculation WITH pre-filtering on the Ck factor, with $\chi ^2$ filter

In [ ]:
# USING THE COMAC MATRIX WITH 15 GROUPS

import calins as cl
import os

# The original formatting of COMAC matrices consists of a folder of files
comac_folder = '/PATH/TO/COMAC/FOLDER/'
case_sensib_file_path = '/PATH/TO/CASE/SENSIB/FILE/case_sensib.15g.sdf'
benchmarks_folder = '/PATH/TO/BENCHMARKS/'

# Necessary: list of paths to benchmark cases
benchmarks_paths = [os.path.join(benchmarks_folder, file) for file in os.listdir(benchmarks_folder) if file.endswith('.sdf')]

# Recommended: Use NDCovariances object
cov = cl.NDCovariances(input_path=comac_folder, format='comac')

# Assimilation with a Chi2 < 1.2 filter: creating a cl.Assimilation object and saving the results
# The 'output_html_path' option triggers the 'export_to_html' function, which saves all GLLSM results and parameters, with details of U238 submatrices ('isotopes_to_detail')
assim = cl.Assimilation(benchmarks_list=benchmarks_paths, 
                        study_case=case_sensib_file_path, 
                        cov_data=cov,
                        reac_list=[2, 4, 16, 18, 102, 103, 452, 1018], 
                        Ck_threshold=0.8, # <-- activation of benchmark case filtering by Ck
                        targetted_chi2=1.2, # <-- activation of Chi2 filtering
                        filtering_chi2_method='diagonal', 
                        output_html_path='/PATH/TO/RESULTS/assim.html', 
                        isotopes_to_detail=['U238'])
print(f"UNC PRIOR: {assim.prior_uncertainty.value} pcm")
print(f"UNC POST: {assim.post_uncertainty.value} pcm")
print(f"BIAS: {assim.bias.value} pcm")
print(f"Keff CASE: {assim.study_case.resp_calc} +/- {assim.study_case.sigma_resp_calc}")

# Save the bias decomposition by iso-reac pair:
#assim.bias.decomposition.to_excel('/PATH/TO/EXCEL/FILE/assim_bias-decomposition_COMAC_15g.xlsx')

# Save the sensitivity histogram of the study case in HTML format
#assim.export_to_html(output_html_path='/PATH/TO/HTML/FILE/assim_case-sensib_15g.html')

# Display benchmark cases excluded by the Chi2 filter (DataFrame containing info on all benchmark cases)
print(assim.bench_list[['PATH', 'REMOVED']])

# Utility functions

In [ ]:
import calins as cl

# Condensing a sensitivity file to a compatible energy mesh - here from 281 groups to 3 groups
case_sensi_file_path = '/PATH/TO/CASE/SENSI/FILE/case_sensi.281g.sdf'
cl.condense_sdf(input_sdf_path=case_sensi_file_path, output_ebins=[20, 1, 0.1, 0], output_sdf_path='/PATH/TO/CASE/SENSI/FILE/case_sensi.condensed.3g.sdf')

# Comparing two cases (sensitivity file or "case" objects)
# Comparison of integral sensitivities for each iso-reac, returns a dataframe/string of discrepancies not covered by "sigma_nb_target" x sigma_integral (statistical disagreement),
# and exceeding a certain percentage "tol_between_sensi_percent" (%)
# Ignores discrepancies where absolute integral sensitivity values represent less than "tol_on_total_sensi_percent" (%) of their total absolute sensitivity
cl.compare_sensitivities(case_1, case_2, sigma_nb_target=4, tol_on_total_sensi_percent=0.1, tol_between_sensi_percent=5.0, results_format="string")


# Computing all similarity indices between two cases

In [ ]:
import calins as cl

# Define paths to sensitivity files or Case objects
case_1_path = '/PATH/TO/CASE1/SENSI/FILE/case1_sensi.sdf'
case_2_path = '/PATH/TO/CASE2/SENSI/FILE/case2_sensi.sdf'
scale_file_path = '/PATH/TO/SCALE/FILE/scale_44g'

# Load cases and covariance data
case_1 = cl.Case(case_1_path)
case_2 = cl.Case(case_2_path) 
cov = cl.NDCovariances(input_path=scale_file_path, format='auto')  # auto-detect format

print(f"Case 1: {case_1.casename}")
print(f"Case 2: {case_2.casename}")

# Calculate E similarity index (normalized dot product)
E_index = cl.calcul_E(case_1, case_2, reac_list=[2, 4, 16, 18, 102, 103, 452, 1018], )
print(f"E similarity index:     {E_index:.4f}")

# Calculate Ck index (weighted by covariances) 
Ck_index = cl.calcul_Ck(case_1, case_2, cov, reac_list=[2, 4, 16, 18, 102, 103, 452, 1018], )
print(f"Ck similarity index:    {Ck_index:.4f}")

# Calculate G overlap index (CEA formula)
G_index = cl.calcul_G(study_case=case_1, bench_case=case_2, reac_list=[2, 4, 16, 18, 102, 103, 452, 1018], )
print(f"G overlap index:        {G_index:.4f}")

# Calculate SSR index (Shared Sensitivity Ratio)
SSR_index = cl.calcul_SSR(study_case=case_1, bench_case=case_2, reac_list=[2, 4, 16, 18, 102, 103, 452, 1018], )
print(f"SSR index:       {SSR_index:.4f}")

# Sorting sensitivities of a case in descending order

In [ ]:
import calins as cl

# Defining the case to analyze using a .sdf file
mycase = "/PATH/TO/CASE/SENSI/FILE/case_sensi.sdf"

# Sorting sensitivities associated with the selected case.
# The analyzed data is integrated (integrate_data=True) over the thermal energy region (energy_region="thermal").
# No lethargy normalization (normalize_lethargy=False). The result is returned as a sorted DataFrame.
# The variables iso_list and reac_list can be used to limit the analysis to certain isotopes and/or reactions
sorted_df = cl.sort_case_sensitivities(
    case=mycase,
    energy_region="thermal",
    integrate_data=True,
    normalize_lethargy=False,
    iso_list=None,
    reac_list=None,
)

# Displaying the first 30 rows of the sorted DataFrame.
cl.display_dataframe(df=sorted_df, max_number_of_rows_displayed=30)

# Searching for cases sensitive to a cross-section of an isotope in a given energy domain

In [ ]:
import calins as cl
import pickle

# Path to a pickle containing a dictionary of Case objects
pickle_db = "/PATH/TO/DATABASE/cases.pickle" 

with open(pickle_db, "rb") as pickle_file:
    dict_pickle = pickle.load(pickle_file)

# Selecting cases to consider for analysis - here all MIX-COMP-THERM except series 006
filtered_cases = cl.filter_cases(dict_pickle, to_keep=["MIX-COMP-THERM"], to_exclude=["MIX-COMP-THERM-006"]) 

# Searching for cases sensitive to the fission cross-sections of U-235 and Pu-239 in the thermal domain,
# based on integral sensitivities, not normalized in lethargy.
# Only bins with a sensitivity ≥ 2e-3, representing more than 10% of the integral sensitivity, are considered sensitive.
sensitive_cases = cl.find_sensitive_cases(
    dict_pickle,
    filtered_cases,
    isotopes=[92235, 94239],
    reactions=[18, 18],
    energy_regions=["thermal", "thermal"],
    sensitivity_threshold=2e-3,
    fraction_threshold = 0.10,
    integrate_data=True,
    normalize_lethargy = False,
)

# Displaying in the terminal the 50 most sensitive cases according to the previous search criteria
cl.display_most_sensitive_cases(dict_sensitive = sensitive_cases, max_number_of_cases_displayed=50)

# Creating a sensitivity heatmap

In [ ]:
import calins as cl
import pickle

# Path to a pickle containing a dictionary of Case objects
pickle_db = "/PATH/TO/DATABASE/cases.pickle" 

with open(pickle_db, "rb") as pickle_file:
    dict_pickle = pickle.load(pickle_file)

# Selecting cases to consider for analysis - here all LEU-COMP-THERM except series 001
list_cases = cl.filter_cases(dict_pickle, to_keep=[], to_exclude=[]) 

# Creating a heatmap of cases sensitive to cross-sections of SCATTER (0), TOTAL (1), ELASTIC (2), N,GAMMA (102)
# Only bins with a sensitivity ≥ 2e-3 and representing more than 10% of the integral sensitivity will be considered sensitive.
cl.plot_sensitivity_heatmap(
    dict_data=dict_pickle,
    cases=list_cases,
    isotopes=[17035, 17037],
    reactions=[0, 1, 2, 102],
    sensitivity_threshold=2e-3,
    fraction_threshold = 0.10,
    user_energy_bins=None,
    title="Cases pickle db sensitivity heatmap for chlorine isotopes",
    namefig=None,
    save_logfile=True,
)